In [1]:
import Eisenia

┌ Info: Recompiling stale cache file /Users/cameronprybol/.julia/compiled/v1.2/Eisenia/cMB3l.ji for Eisenia [ff91eec8-cf20-4569-aeea-763e189e8d12]
└ @ Base loading.jl:1240


In [2]:
import BioSequences

In [3]:
import Random

In [92]:
import Dates
import LightGraphs
import Statistics

In [5]:
LENGTH = 4

4

In [6]:
Random.seed!(LENGTH)

Random.MersenneTwister(UInt32[0x00000004], Random.DSFMT.DSFMT_state(Int32[-1125970028, 1072950084, 1987879523, 1073737715, 1318759160, 1073082403, -665387610, 1073002484, 775432869, 1072783382  …  -1193370493, 1072918807, 503071495, 1073572056, -1375397214, 586182507, 2038247757, 798803726, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000

In [7]:
seq = BioSequences.randdnaseq(LENGTH)

4nt DNA Sequence:
GTTT

In [8]:
id = Random.randstring(LENGTH)

"QTGJ"

In [9]:
record = BioSequences.FASTA.Record(id, seq)

BioSequences.FASTA.Record:
   identifier: QTGJ
  description: <missing>
     sequence: GTTT

In [10]:
canonical_kmers =
    sort(
    unique([BioSequences.canonical(kmer) for (index, kmer) in
            BioSequences.each(BioSequences.DNAKmer{3}, BioSequences.FASTA.sequence(record))])
    )

2-element Array{BioSequences.Kmer{BioSymbols.DNA,3},1}:
 AAA
 AAC

In [11]:
observations = [record]

1-element Array{BioSequences.FASTA.Record,1}:
 BioSequences.FASTA.Record:
   identifier: QTGJ
  description: <missing>
     sequence: GTTT

In [12]:
?Eisenia.build_stranded_kmer_graph

```
build_stranded_kmer_graph(canonical_kmers, observations)
```

Create a weighted, strand-specific kmer (de bruijn) graph from a set of kmers and a series of sequence observations in FASTA format.


In [13]:
stranded_kmer_graph = Eisenia.build_stranded_kmer_graph(canonical_kmers, observations)

{4, 1} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [14]:
filename = "$(BioSequences.FASTA.identifier(record)).strand_specific.svg"
Eisenia.plot_stranded_kmer_graph(stranded_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"QTGJ.strand_specific.svg\" width=50%>\n")

I don't think we should normalize, it doesn't seem necessary. Instead, we should always build the stranded kmer graph, and plot that. The stranded kmer graph is used for viterbi. Canonical kmer graph is undirected. Collapse down the directed graph, retaining read and strand info. Use simple graph structure to determine reference path and call variants. Use dominant orientation to determine which orientation to report.

In [15]:
canonical_kmer_graph = Eisenia.build_canonical_kmer_graph(canonical_kmers, observations)

{2, 1} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [16]:
filename = "$(BioSequences.FASTA.identifier(record)).canonical.svg"
Eisenia.plot_canonical_kmer_graph(canonical_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"QTGJ.canonical.svg\" width=50%>\n")

In [108]:
# return the dominant path(s) as fasta with coverage info
for (i, connected_component) in enumerate(LightGraphs.connected_components(canonical_kmer_graph))
    maximum_weight = maximum(v -> length(canonical_kmer_graph.vprops[v][:coverage]), connected_component)
    @show maximum_weight
    maximum_weight_indices = findall(v -> length(canonical_kmer_graph.vprops[v][:coverage]) == maximum_weight, connected_component)
    @show maximum_weight_indices
    initial_vertex = rand(maximum_weight_indices)
    @show initial_vertex
    true_walk = []
    current_vertex = initial_vertex
    for coverage in canonical_kmer_graph.vprops[initial_vertex][:coverage]
        (o, (oi, oo)) = coverage
        @show coverage
        if oo
            next_step = o => (oi+1 => oo)
        else
            next_step = o => (oi-1 => oo)
        end
        @show next_step
        for neighbor in LightGraphs.neighbors(canonical_kmer_graph, current_vertex)
            if next_step in canonical_kmer_graph.vprops[neighbor][:coverage]
                push!(true_walk, (neighbor => true))
            end
        end
    end
    @show true_walk
    false_walk = []
    current_vertex = initial_vertex
    for coverage in canonical_kmer_graph.vprops[initial_vertex][:coverage]
        (o, (oi, oo)) = coverage
        @show coverage
        if oo
            next_step = o => (oi-1 => oo)
        else
            next_step = o => (oi+1 => oo)
        end
        @show next_step
        for neighbor in LightGraphs.neighbors(canonical_kmer_graph, current_vertex)
            if next_step in canonical_kmer_graph.vprops[neighbor][:coverage]
                push!(false_walk, (neighbor => false))
            end
        end
    end
    @show false_walk
    reverse_complemented_false_walk = [vertex => !orientation for (vertex, orientation) in false_walk[end:-1:1]]
    @show reverse_complemented_false_walk
    @show path = [reverse_complemented_false_walk..., initial_vertex => true, true_walk...]
    (vertex, orientation) = first(path)
    kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
    if !orientation
        kmer = BioSequences.reverse_complement(kmer)
    end
    initial_sequence = BioSequences.DNASequence(kmer)
    @show initial_sequence
    path_sequence = initial_sequence
    for (vertex, orientation) in path[2:end]
        kmer = canonical_kmer_graph.gprops[:canonical_kmers][vertex]
        if !orientation
            kmer = BioSequences.reverse_complement(kmer)
        end
        sequence = BioSequences.DNASequence(kmer)
        @show sequence
        @show sequence_prefix = sequence[1:end-1]
        k = length(sequence)
        @show path_sequence_suffix = path_sequence[end-(k-2):end]
        @assert sequence_prefix == path_sequence_suffix
        push!(path_sequence, sequence[end])
    end
    @show path_sequence
    path_coverage = [length(canonical_kmer_graph.vprops[vertex][:coverage]) for (vertex, orientation) in path]
    @show path_coverage
    @show Statistics.mean(path_coverage)
    @show maximum(path_coverage)
    @show minimum(path_coverage)
    @show Statistics.middle(path_coverage)
    @show Statistics.median(path_coverage)
    @show Statistics.std(path_coverage)
end

maximum_weight = 1
maximum_weight_indices = [1, 2]
initial_vertex = 1
coverage = 1 => (2 => false)
next_step = 1 => (1 => false)
true_walk = Any[2 => true]
coverage = 1 => (2 => false)
next_step = 1 => (3 => false)
false_walk = Any[]
reverse_complemented_false_walk = Pair[]
path = [reverse_complemented_false_walk..., initial_vertex => true, true_walk...] = Pair{Int64,Bool}[1 => 1, 2 => 1]
initial_sequence = AAA
sequence = AAC
sequence_prefix = sequence[1:end - 1] = AA
path_sequence_suffix = path_sequence[end - (k - 2):end] = AA
path_sequence = AAAC
path_coverage = [1, 1]
Statistics.mean(path_coverage) = 1.0
maximum(path_coverage) = 1
minimum(path_coverage) = 1
Statistics.middle(path_coverage) = 1.0
Statistics.median(path_coverage) = 1.0
Statistics.std(path_coverage) = 0.0


In [ ]:
# return alternate paths along dominant path(s) with coverage info as VCF
# not applicable here